In [5]:
#################################################
#created the 17/05/2018 14:14 by Alexis Blanchet#
#################################################
#-*- coding: utf-8 -*-
'''

'''

'''
Améliorations possibles:

'''
############ display options ####################
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

import pandas as pd
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
#################################################
###########        Imports      #################
#################################################
import pandas as pd
import numpy as np

In [18]:
PTV = pd.read_csv('IPTV_2018-05-09_TF1.csv')[['TITRE','DUREE','description programme','HEURE',
       'debut']]
PTV['fin'] = PTV['debut']+PTV['DUREE']
Points = pd.read_csv('merged_2018-05-09.csv')[['minutes']]



In [19]:
PTV

,TITRE,DUREE,description programme,HEURE,debut,fin
0,TFou,120,Magazine jeunesse,06:25:00,385,505
1,Météo,5,Météo,08:25:00,505,510
2,Téléshopping,45,Magazine de télé-achat,08:30:00,510,555
3,Météo,5,Météo,09:15:00,555,560
4,Petits secrets entre voisins,30,Série réaliste,09:20:00,560,590
5,Petits secrets entre voisins,35,Série réaliste,09:50:00,590,625
6,Demain nous appartient,35,Série dramatique,10:25:00,625,660
7,Les feux de l'amour,55,Feuilleton sentimental,11:00:00,660,715
8,Petits plats en équilibre,5,Magazine de la gastronomie,11:55:00,715,720
9,Les douze coups de midi,55,Jeu,12:00:00,720,775


In [66]:
Points.head()

,minutes
0,394
1,437
2,485
3,500
4,504


In [48]:
begin = (pd.DataFrame(PTV.loc[(PTV.shape[0]-1)]).T).reset_index().drop(['index'],axis=1)

In [49]:
begin

,TITRE,DUREE,description programme,HEURE,debut,fin
0,Programmes de la nuit,255,Programme indéterminé,02:10:00,130,385


In [174]:


verbose = True
label = []
#Initialisation des Variables
index_CP = 0
index_PTV = PTV.shape[0]-1
##############
lastCP = 200
lastPub= 0
lastend = 130
lastduree = 255
planifiedend = 385
begin = True
#Initialisation du NewPTV
OnlinePTV = pd.DataFrame()
OnlinePTV['minute'] = [180]
OnlinePTV['TITRE'] = 'Programmes de la nuit'
OnlinePTV['Change Point'] = 'Non'
OnlinePTV['pourcentage de la durée'] = 0
OnlinePTV['Évenement'] = 'Début de Détection'
OnlinePTV


,minute,TITRE,Change Point,pourcentage de la durée,Évenement
0,180,Programmes de la nuit,Non,0,Début de Détection


In [175]:


for i in range(180,1620):
    #incrémentation
    lastPub+=1
    lastCP+=1
    if(i%1440==Points['minutes'][index_CP]):
        if(lastCP<4):
            if(verbose):
                print("Détection multiple d'un même événement")
            index_CP+=1
            label.append(label[-1])
            continue
        else:
            percentage = (i%1440-lastend)/lastduree
            if(i%1440-lastend<0):
                percentage = (i-lastend)/lastduree
            if(planifiedend>i):
                #On détermine la place du CP dans le programme
                if(lastduree<15):
                    if(verbose):
                        print("Change Point en fin de programme")
                    if((i-lastend)<10):
                        OnlinePTV.loc[OnlinePTV.shape[0]] = [i%1440,PTV['TITRE'].iloc[index_PTV],'oui',percentage,"fin d'un programme court"]
                        lastend = i
                        lastCP=0
                        index_PTV += 1
                        index_PTV = index_PTV%31
                        lastduree = PTV['DUREE'].iloc[index_PTV]
                        planifiedend = (lastend + lastduree)
                        label.append(2)
                else:
                    if(percentage<=0.25):
                        if(verbose):
                            print("Change Point en début de programme")
                        if(lastPub>20):
                            OnlinePTV.loc[OnlinePTV.shape[0]] = [i%1440,"publicité",'oui',percentage,"publicité dans la première partie d'un programme"]
                            label.append(1)
                            lastCP=0
                            lastPub = 0
                        else:
                            if(verbose):
                                print("Faux Change Point en début de programme")
                            label.append(0)


                    elif(percentage>0.8):
                        if(verbose):
                            print("Change Point en fin de programme")
                        if((i-lastend)<10):
                            OnlinePTV.loc[OnlinePTV.shape[0]] = [i%1440,PTV['TITRE'].iloc[index_PTV],'oui',percentage,"fin d'un programme court"]
                            lastend = i
                            lastCP=0
                            index_PTV += 1
                            index_PTV = index_PTV%31
                            lastduree = PTV['DUREE'].iloc[index_PTV]
                            planifiedend = (lastend + lastduree)
                            label.append(2)
                        elif(planifiedend-i<10):
                            OnlinePTV.loc[OnlinePTV.shape[0]] = [i%1440,PTV['TITRE'].iloc[index_PTV],'oui',percentage,"fin d'un programme long"]
                            lastend = i
                            lastCP=0
                            index_PTV += 1  
                            index_PTV = index_PTV%31
                            lastduree = PTV['DUREE'].iloc[index_PTV]
                            planifiedend = (lastend + lastduree)
                            label.append(2)
                        elif(lastPub>30):
                            OnlinePTV.loc[OnlinePTV.shape[0]] = [i%1440,"publicité",'oui',percentage,"publicité dans la première partie d'un programme"]
                            lastCP=0
                            lastPub = 0
                            label.append(1)
                        else:
                            if(verbose):
                                print("Faux Change Point en Début de Programme")
                            label.append(0)
                        
                
                    else:
                        if(verbose):
                            print("Change Point en milieu de programme")
                        if(lastPub>20):
                            OnlinePTV.loc[OnlinePTV.shape[0]] = [i%1440,"publicité",'oui',percentage,"publicité au milieu d'un programme"]
                            lastCP=0
                            lastPub = 0
                            label.append(1)
                        else:
                            if(verbose):
                                print("Faux Change Point en début de programme")
                            label.append(0)

                    
                
            
            
            elif(planifiedend<i):
                if(verbose):
                    print("Change Point à l'éxtérieur du programme en cours")
                if(i>lastend+lastduree+(min(20,max(0.2*lastduree,5)))):
                    OnlinePTV.loc[OnlinePTV.shape[0]] = [i%1440,PTV['TITRE'].iloc[index_PTV],'oui',percentage,"fin non détectée d'un programme"]
                    lastend = planifiedend
                    lastCP=0
                    index_PTV += 1  
                    index_PTV = index_PTV%31
                    lastduree = PTV['DUREE'].iloc[index_PTV]
                    planifiedend = (lastend + lastduree)
                elif(i<=lastend+int(1.2*lastduree)+1):
                    if(lastduree<15):
                        OnlinePTV.loc[OnlinePTV.shape[0]] = [i%1440,PTV['TITRE'].iloc[index_PTV],'oui',percentage,"fin d'un programme court"]
                        lastend = i
                        lastCP=0
                        index_PTV += 1  
                        index_PTV = index_PTV%31
                        lastduree = PTV['DUREE'].iloc[index_PTV]
                        planifiedend = (lastend + lastduree)
                    else:   
                        OnlinePTV.loc[OnlinePTV.shape[0]] = [i%1440,PTV['TITRE'].iloc[index_PTV],'oui',percentage,"fin d'un programme long"]
                        lastend = i+3
                        lastCP=0
                        index_PTV += 1  
                        index_PTV = index_PTV%31
                        lastduree = PTV['DUREE'].iloc[index_PTV]
                        planifiedend = (lastend + lastduree)
                    
                
            
        
        
        
            index_CP+=1
            
    else:
        percentage = (i%1440-lastend)/lastduree
        if(i%1440-lastend<0):
            percentage = (i-lastend)/lastduree
        if(lastduree<15 and i>lastend+lastduree):
            if(verbose):
                print("fin non détectée d'un programme (rattrapage hors CP)")
            OnlinePTV.loc[OnlinePTV.shape[0]] = [i%1440,PTV['TITRE'].iloc[index_PTV],'non',percentage,"fin non détectée d'un programme court(rattrapage hors CP)"]
            lastend = planifiedend+3
            lastCP=0
            index_PTV += 1  
            index_PTV = index_PTV%31
            lastduree = PTV['DUREE'].iloc[index_PTV]
            planifiedend = (lastend + lastduree)
            
        if(i>lastend+lastduree+(min(20,max(0.2*lastduree,5)))):
            if(verbose):
                print("fin non détectée d'un programme (rattrapage hors CP)")
            OnlinePTV.loc[OnlinePTV.shape[0]] = [i%1440,PTV['TITRE'].iloc[index_PTV],'non',percentage,"fin non détectée d'un programme long(rattrapage hors CP)"]
            lastend = planifiedend+3
            lastCP=0
            index_PTV += 1  
            index_PTV = index_PTV%31
            lastduree = PTV['DUREE'].iloc[index_PTV]
            planifiedend = (lastend + lastduree)
        else:
            pass
            #regular point; we don't do a thing
        
        
        

    

Change Point à l'éxtérieur du programme en cours
Change Point en milieu de programme
Change Point en milieu de programme
Change Point en fin de programme
Faux Change Point en Début de Programme
Change Point en fin de programme
Faux Change Point en Début de Programme
Change Point en fin de programme
Faux Change Point en Début de Programme
Change Point en fin de programme
Change Point en fin de programme
Change Point en début de programme
Change Point en milieu de programme
Faux Change Point en début de programme
Change Point en milieu de programme
Change Point en fin de programme
Détection multiple d'un même événement
fin non détectée d'un programme (rattrapage hors CP)
Change Point en milieu de programme
fin non détectée d'un programme (rattrapage hors CP)
Change Point en milieu de programme
Change Point en milieu de programme
Faux Change Point en début de programme
Change Point à l'éxtérieur du programme en cours
Change Point en milieu de programme
Détection multiple d'un même événeme

In [176]:
OnlinePTV['Heure'] = OnlinePTV['minute'].apply(lambda x: str(int(x/60))+':'+str(x%60))
display_side_by_side(PTV,OnlinePTV)

,TITRE,DUREE,description programme,HEURE,debut,fin
0,TFou,120,Magazine jeunesse,06:25:00,385,505
1,Météo,5,Météo,08:25:00,505,510
2,Téléshopping,45,Magazine de télé-achat,08:30:00,510,555
3,Météo,5,Météo,09:15:00,555,560
4,Petits secrets entre voisins,30,Série réaliste,09:20:00,560,590
5,Petits secrets entre voisins,35,Série réaliste,09:50:00,590,625
6,Demain nous appartient,35,Série dramatique,10:25:00,625,660
7,Les feux de l'amour,55,Feuilleton sentimental,11:00:00,660,715
8,Petits plats en équilibre,5,Magazine de la gastronomie,11:55:00,715,720
9,Les douze coups de midi,55,Jeu,12:00:00,720,775
